# Setup

In [266]:

import pandas as pd;

transactions = pd.read_csv("./financial_transactions/transactions_data.csv")
transactions.set_index("id")


,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
id,,,,,,,,,,,
7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
...,...,...,...,...,...,...,...,...,...,...,...
7594787,2010-01-31 14:34:00,1916,3305,$29.49,Swipe Transaction,54850,Panama City,FL,32401.0,4814,NaN
7594788,2010-01-31 14:35:00,364,4623,$13.39,Swipe Transaction,83480,Alcoa,TN,37701.0,9402,NaN
7594790,2010-01-31 14:35:00,452,4249,$60.39,Swipe Transaction,36934,Houston,TX,77096.0,7538,NaN


# Présentation des données transactions
Nous intérprétons les données suivantes:
- client_id comme Client
- merchant_xxx comme Magasin

NB: La colonne 'mcc' représente des codes correspondant à la catégorie de l'achat. Ces codes sont repris dans le fichier `mcc_codes.json`.
Son utilisation ici ne sera pas très pertinente, nous ignorerons cette colonne

In [267]:
transactions.groupby('client_id').client_id.count() # 1083 clients uniques


client_id
0        95
1        83
2        95
3        41
4       116
       ... 
1993     72
1995     98
1996     90
1997     90
1998     32
Name: client_id, Length: 1083, dtype: int64

In [268]:
# Classement de la plus grosse transaction par magasin et par état

transactions.groupby(["merchant_state", "merchant_id"]).apply(lambda df: df.loc[df.amount.idxmax()]).amount
# On groupe les états et les identifiants des magasins, et on trie le montant de chaque transaction (ASC)
# Et on y lie le montant correspondant.

C:\Users\Hugo\AppData\Local\Temp\ipykernel_18352\1003607640.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transactions.groupby(["merchant_state", "merchant_id"]).apply(lambda df: df.loc[df.amount.idxmax()]).amount


merchant_state  merchant_id
AK              7257           $1234.77
                15574           $467.00
                44795            $73.82
AL              634              $25.50
                687               $1.70
                                 ...   
WY              78454            $81.68
                85797            $47.72
                86438             $1.42
                90461             $8.85
                90709            $79.80
Name: amount, Length: 13956, dtype: object

In [269]:
# Le nombre de transactions, le montant minimum et maximum, par état
transactions.groupby(["merchant_state"]).amount.agg([len, min, max])

C:\Users\Hugo\AppData\Local\Temp\ipykernel_18352\3133034948.py:2: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  transactions.groupby(["merchant_state"]).amount.agg([len, min, max])
C:\Users\Hugo\AppData\Local\Temp\ipykernel_18352\3133034948.py:2: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  transactions.groupby(["merchant_state"]).amount.agg([len, min, max])


,len,min,max
merchant_state,,,
AK,48,$-104.00,$73.82
AL,1313,$-149.00,$99.86
AR,806,$-100.00,$99.00
AZ,1425,$-100.00,$99.64
CA,10802,$-100.00,$99.93
...,...,...,...
Vietnam,11,$-219.00,$264.37
WA,2301,$-100.00,$99.96
WI,1435,$-100.00,$99.73


# Traitement des valeurs nulles
Lorsqu'une transaction est faite en ligne, le code postal et l'état du magasin ne peuvent pas exister.
Nous allons donc remplacer ces cellules vides pour une meilleure visibilité et un futur traitement de ces données.

Aussi, certaines transactions peuvent être erronées. Celles qui ne le sont pas auront la mention "No error"


In [270]:
# Certaines valeurs sont nulles lorsque 'use_chip' est égal à 'Online Transaction'. C'est le cas de merchant_state et zip.
transactions[pd.isnull(transactions.merchant_state)]


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,$26.46,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,$261.58,Online Transaction,50292,ONLINE,NaN,NaN,7801,NaN
18,7475346,2010-01-01 00:34:00,394,4717,$26.04,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
24,7475353,2010-01-01 00:43:00,301,3742,$10.17,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
26,7475356,2010-01-01 00:45:00,566,3439,$16.86,Online Transaction,16798,ONLINE,NaN,NaN,4121,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99940,7594723,2010-01-31 14:18:00,94,2890,$172.13,Online Transaction,71397,ONLINE,NaN,NaN,6300,NaN
99947,7594730,2010-01-31 14:20:00,828,4235,$10.26,Online Transaction,16798,ONLINE,NaN,NaN,4121,NaN
99971,7594758,2010-01-31 14:27:00,1909,1136,$37.87,Online Transaction,15143,ONLINE,NaN,NaN,4784,NaN
99993,7594785,2010-01-31 14:34:00,1194,4614,$24.00,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN


In [271]:
# Nous remplaçons ces cellules vides par une information plus pertinente
transactions.merchant_state = transactions.merchant_state.fillna("Online")
transactions.zip = transactions.zip.fillna("Unknown")

# Vérifions bien que les états ne sont plus nuls.
[transactions.merchant_state.isnull().sum(), 
transactions.zip.isnull().sum()]


[np.int64(0), np.int64(0)]

In [272]:
# Message d'erreur par défaut
transactions.errors = transactions.errors.fillna("No error")

In [273]:
# Présentation des données nettoyées des valeurs nulles.
# On voit bien les modifications appliquées sur les colonnes merchant_state, zip, et errors
transactions.loc[transactions.use_chip == "Online Transaction"].head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,$26.46,Online Transaction,39021,ONLINE,Online,Unknown,4784,No error
8,7475336,2010-01-01 00:21:00,335,5131,$261.58,Online Transaction,50292,ONLINE,Online,Unknown,7801,No error
18,7475346,2010-01-01 00:34:00,394,4717,$26.04,Online Transaction,39021,ONLINE,Online,Unknown,4784,No error
24,7475353,2010-01-01 00:43:00,301,3742,$10.17,Online Transaction,39021,ONLINE,Online,Unknown,4784,No error
26,7475356,2010-01-01 00:45:00,566,3439,$16.86,Online Transaction,16798,ONLINE,Online,Unknown,4121,No error


# Nettoyage des données
Maintenant que les données sont complètes, il faut les rendres plus simple à manipuler.

C'est notamment le cas de la colonne "amount", qui est actuellement une chaîne de caractères, car elle commence par le symbole monétaire.
Bien que cela reste lisible pour nous, cette colonne ne peut pas être manipulée pour des chiffres.

Par exemple, on peut voir ci-dessous que la plus grande valeur (**top**) est $80.00, malgré la présences de transactions dépassant les $1000.
Mais, si l'on compare caractère par caractère, $1 est plus petit que $8, donc c'est $80.00 qui sortira vainqueur de cette comparaison.

In [274]:
transactions.amount.describe()

count     100000
unique     16770
top       $80.00
freq        1054
Name: amount, dtype: object

In [275]:
# On convertit la colonne en valeur numérique, en retirant le premier caractère de la colonne 'amount'
transactions.amount = pd.to_numeric(transactions.amount.str[1:])
transactions.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,No error
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,No error
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,No error
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,No error
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,No error


### Actuellement, la colonne **date** est considérée comme un string par pandas.
On va la convertir en date pour pouvoir la manipuler efficacement.

In [276]:
transactions["parsed_date"] = pd.to_datetime(transactions.date, format="%Y-%m-%d %H:%M:%S")
transactions["parsed_date"].head()

0   2010-01-01 00:01:00
1   2010-01-01 00:02:00
2   2010-01-01 00:02:00
3   2010-01-01 00:05:00
4   2010-01-01 00:06:00
Name: parsed_date, dtype: datetime64[ns]

### Suppression des doublons
Nous supposons qu'un client ne peut faire 2 transactions au même moment avec la même carte, nous allons les retirer.


In [277]:
transactions_before_removal = transactions;
transactions = transactions.drop_duplicates(subset=['parsed_date', 'client_id', 'card_id'], keep="last")

# On peut observer une suppression de 593 doublons
pd.Series([transactions_before_removal.shape[0], transactions.shape[0]])


0    100000
1     99407
dtype: int64